In [1]:
import sys
import os
import pickle
import torch

# Manually specify the project root if __file__ is not available
project_root = os.path.abspath("..")  # Adjust this path if needed
sys.path.append(project_root)

from GPT_Tokenizer.BPE_Tokenizer import BPETokenizer

In [2]:
# read data to inspect

file_path = r'data\\cleaned_combined_books.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(f"length of dataset in chars: {len(text)}")

length of dataset in chars: 2661874


In [4]:
# here are all the unique characters occures in the text

unique_chars = sorted(list(set(text)))

print(f'vocab size: {len(unique_chars)}')
print(f"unique chars: {''.join(unique_chars)}")

vocab size: 72
unique chars: 
 !"',.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [5]:
# Process to train the tokenizer to get the mappings

# tokenizer = BPETokenizer(text, iterations=50)

# tokens, idx_to_char, char_to_idx = tokenizer.bpe()

# # Store the mappings for future use
# with open("tokenizer_mappings.pkl", "wb") as f:
#     pickle.dump({
#         "char_to_idx": char_to_idx,
#         "idx_to_char": idx_to_char
#     }, f)

In [20]:
# reutilize the mappings from tokenizer

with open("tokenizer_mappings.pkl", "rb") as f:
    mappings = pickle.load(f)

char_to_idx = mappings["char_to_idx"]
idx_to_char = mappings["idx_to_char"]
vocab_size = len(char_to_idx)

tokenizer = BPETokenizer(text, iterations=50)

print(tokenizer.encode("hii there", char_to_idx))
print(tokenizer.decode(tokenizer.encode("hii there", char_to_idx), idx_to_char))

[23, 46, 46, 19, 25, 23, 24, 61, 24]
hii there


In [7]:
# let's now encode entire dataset and store them in torch tensor

data = torch.tensor(tokenizer.encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([2661874]) torch.int64


In [13]:
# split data in train and validation

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8

train_data[:block_size+1]

tensor([67, 33, 52, 28, 45, 40, 52, 33, 20])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"input: {context} | target: {target}")

input: tensor([67]) | target: 33
input: tensor([67, 33]) | target: 52
input: tensor([67, 33, 52]) | target: 28
input: tensor([67, 33, 52, 28]) | target: 45
input: tensor([67, 33, 52, 28, 45]) | target: 40
input: tensor([67, 33, 52, 28, 45, 40]) | target: 52
input: tensor([67, 33, 52, 28, 45, 40, 52]) | target: 33
input: tensor([67, 33, 52, 28, 45, 40, 52, 33]) | target: 20


In [19]:
torch.manual_seed(1337)
batch_size = 4 # independent sequence processing in parallel
block_size = 8 # max context length of predictions

def get_batch(split):
    # generate batch of data from inputs x and target y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

xb, yb = get_batch('train')

print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

print('---------------')

for b in range(batch_size):             # batch dimension
    for t in range(block_size):         # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"context: {context.tolist()} | target: {target}")

inputs
torch.Size([4, 8])
tensor([[ 7, 34,  2, 21, 52, 33, 34,  2],
        [54, 35, 35, 24, 30, 34,  2, 54],
        [54, 35, 40,  2, 55, 71, 52, 71],
        [52,  9, 40, 52, 54, 33, 33, 34]])
targets
torch.Size([4, 8])
tensor([[34,  2, 21, 52, 33, 34,  2, 21],
        [35, 35, 24, 30, 34,  2, 54, 28],
        [35, 40,  2, 55, 71, 52, 71, 56],
        [ 9, 40, 52, 54, 33, 33, 34, 59]])
---------------
context: [7] | target: 34
context: [7, 34] | target: 2
context: [7, 34, 2] | target: 21
context: [7, 34, 2, 21] | target: 52
context: [7, 34, 2, 21, 52] | target: 33
context: [7, 34, 2, 21, 52, 33] | target: 34
context: [7, 34, 2, 21, 52, 33, 34] | target: 2
context: [7, 34, 2, 21, 52, 33, 34, 2] | target: 21
context: [54] | target: 35
context: [54, 35] | target: 35
context: [54, 35, 35] | target: 24
context: [54, 35, 35, 24] | target: 30
context: [54, 35, 35, 24, 30] | target: 34
context: [54, 35, 35, 24, 30, 34] | target: 2
context: [54, 35, 35, 24, 30, 34, 2] | target: 54
context: [5

### Bi-gram language model

In [24]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C) | Batch * Context Window * Vocab Size

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self(idx)

            # focus only on the last time step
            logits = logits[:, -1, :]

            # apply softmax to get the probabilities
            probs = F.softmax(logits, dim=1)

            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(tokenizer.decode(m.generate(idx, max_new_tokens=100)[0].tolist(), idx_to_char))

torch.Size([32, 122]) tensor(5.3654, grad_fn=<NllLossBackward0>)
wdFou.S t w, enrel1aZkCres25y st1der Co ,d ;t eling'.om to ?BNof L9waTe isKstJ.,d BNMld d uN and OQy of ffkTKkomYel6W?CaanatouedADkha"sBws of AtC ais


[Click here](https://www.youtube.com/watch?v=kCc8FmEb1nY&t=2791s)